In [1]:
import pennylane as qml
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVC
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [21]:
mnist = fetch_openml('mnist_784', version=1)
X, y = mnist.data, mnist.target

In [22]:
X = X[(y == '1') | (y == '3') | (y == '5') | (y == '7')]
y = y[(y == '1') | (y == '3') | (y == '5') | (y == '7')]

map = {'1': 0, '3': 1, '5': 2, '7': 3}
y = y.map(map)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [24]:
X_train_torch = torch.tensor(X_train, dtype=torch.float32)
y_train_torch = torch.tensor(y_train.to_numpy(), dtype=torch.long)
X_test_torch  = torch.tensor(X_test,  dtype=torch.float32)
y_test_torch  = torch.tensor(y_test.to_numpy(),  dtype=torch.long)

train_dataset = TensorDataset(X_train_torch, y_train_torch)
test_dataset  = TensorDataset(X_test_torch,  y_test_torch)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_dataset,  batch_size=32, shuffle=False)

In [25]:
n_qubits = 8
n_layers = 1

dev = qml.device('lightning.gpu', wires=n_qubits)

@qml.qnode(dev, interface="torch")
def qlayer(inputs, weights):
    qml.AngleEmbedding(inputs, wires=list(np.arange(n_qubits)), rotation='X')
    qml.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    return tuple(qml.expval(qml.PauliZ(i)) for i in range(n_qubits))

In [26]:
X_train_sampled = []
y_train_sampled = []


for class_label in np.unique(y_train):
    class_indices = np.where(y_train == class_label)[0]
    sampled_indices = np.random.choice(class_indices, size=5, replace=False)

    X_train_sampled.extend(X_train[sampled_indices])
    y_train_sampled.extend(y_train.iloc[sampled_indices]) 

X_train_sampled = np.array(X_train_sampled)
y_train_sampled = np.array(y_train_sampled)

X_train_sampled_torch = torch.tensor(X_train_sampled, dtype=torch.float32)
y_train_sampled_torch = torch.tensor(y_train_sampled, dtype=torch.long)

train_sampled_dataset = TensorDataset(X_train_sampled_torch, y_train_sampled_torch)
train_sampled_loader = DataLoader(train_sampled_dataset, batch_size=32, shuffle=True)

In [27]:
class HybridClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(len(X.T), 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 8)
        self.scale = nn.Parameter(torch.tensor([2 * np.pi]))
        self.qnn_weights = nn.Parameter(torch.rand(n_layers, n_qubits, 3) * 1e-3)
        self.output_layer = nn.Linear(n_qubits, 4)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = torch.tanh(self.fc4(x)) * self.scale
        batch_size = x.size(0)
        out = torch.empty((batch_size, n_qubits), dtype=torch.float32, device=x.device)
        for batch_idx in range(batch_size):
            expvals = qlayer(x[batch_idx], self.qnn_weights)
            out[batch_idx] = torch.stack(expvals)
        x = self.output_layer(out)
        return x

In [28]:
class ClassicalClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(len(X.T), 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 8)
        self.output_layer = nn.Linear(8, 4)  

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.output_layer(x)
        return x

In [33]:
model = HybridClassifier()

In [34]:
learning_rate = 1e-3
batch_size = 32
epochs = 30

torch_device = "cuda" if torch.cuda.is_available() else "cpu"

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    model.to(torch_device)  
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(torch_device), y.to(torch_device)
        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch+1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}")


def test_loop(dataloader, model, loss_fn):
    model.eval()
    num_batches = len(dataloader)
    test_loss = 0

    pred = None
    X = None
    correct = 0
    model.to(torch_device)  
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(torch_device), y.to(torch_device)
            pred = model(X)
            predicted_label = torch.argmax(pred, dim=1)
            correct += torch.count_nonzero(predicted_label == y)
            test_loss += loss_fn(pred, y).item()

    print(f"Accuracy: {correct / (len(dataloader) * batch_size) * 100}")
    test_loss /= num_batches
    print(f"Avg loss: {test_loss:>8f} \n")

    return test_loss

In [35]:
import time

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5)
start_time = time.time()

print(f"Before training sampled data\n--------------------------")
for t in range(epochs):
    print(f"Epoch {t+1}\n--------------------------")
    train_loop(train_sampled_loader, model, loss_fn, optimizer)
    test_loss = test_loop(test_loader, model, loss_fn)
    scheduler.step(test_loss)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Training took {elapsed_time/60:.2f} minutes.")
test_loop(test_loader, model, loss_fn)
print("Done!")

Before training sampled data
--------------------------
Epoch 1
--------------------------
loss: 1.430017 [   20/   20
Accuracy: 27.688547134399414
Avg loss: 1.385370 

Epoch 2
--------------------------
loss: 1.312480 [   20/   20
Accuracy: 28.282121658325195
Avg loss: 1.370455 

Epoch 3
--------------------------
loss: 1.236703 [   20/   20
Accuracy: 29.1201114654541
Avg loss: 1.358579 

Epoch 4
--------------------------
loss: 1.189448 [   20/   20
Accuracy: 30.045391082763672
Avg loss: 1.347428 

Epoch 5
--------------------------
loss: 1.151500 [   20/   20
Accuracy: 32.01815414428711
Avg loss: 1.337312 

Epoch 6
--------------------------
loss: 1.114667 [   20/   20
Accuracy: 33.99092102050781
Avg loss: 1.328932 

Epoch 7
--------------------------
loss: 1.089090 [   20/   20
Accuracy: 35.26536178588867
Avg loss: 1.320934 

Epoch 8
--------------------------
loss: 1.066827 [   20/   20
Accuracy: 35.701812744140625
Avg loss: 1.313004 

Epoch 9
--------------------------
loss: 1.04

In [32]:
import time

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5)

start_time = time.time()

print(f"Before training\n--------------------------")
for t in range(epochs):
    print(f"Epoch {t+1}\n--------------------------")
    train_loop(train_loader, model, loss_fn, optimizer)
    test_loss = test_loop(test_loader, model, loss_fn)
    scheduler.step(test_loss)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Training took {elapsed_time/60:.2f} minutes.")
test_loop(test_loader, model, loss_fn)
print("Done!")

Before training
--------------------------
Epoch 1
--------------------------
loss: 1.108349 [   32/22899
loss: 0.171429 [ 3232/22899
loss: 0.079054 [ 6432/22899
loss: 0.031366 [ 9632/22899
loss: 0.020665 [12832/22899
loss: 0.150173 [16032/22899
loss: 0.128595 [19232/22899
loss: 0.007454 [22432/22899
Accuracy: 97.46857452392578
Avg loss: 0.106847 

Epoch 2
--------------------------
loss: 0.134964 [   32/22899
loss: 0.012960 [ 3232/22899
loss: 0.097291 [ 6432/22899
loss: 0.006723 [ 9632/22899
loss: 0.002523 [12832/22899
loss: 0.105229 [16032/22899
loss: 0.208252 [19232/22899
loss: 0.009216 [22432/22899
Accuracy: 98.09706115722656
Avg loss: 0.086666 

Epoch 3
--------------------------
loss: 0.015706 [   32/22899
loss: 0.008369 [ 3232/22899
loss: 0.008434 [ 6432/22899
loss: 0.002029 [ 9632/22899
loss: 0.000647 [12832/22899
loss: 0.026236 [16032/22899
loss: 0.013096 [19232/22899
loss: 0.002438 [22432/22899
Accuracy: 98.1668930053711
Avg loss: 0.126141 

Epoch 4
--------------------------